# 02_clean_features

This notebook builds a clean daily features table from master hydrology/load data.

**Outputs:** `features_daily.csv`

Steps:
1. Load master file
2. Normalize column names & types
3. Fill small gaps conservatively
4. Add calendar features
5. Add lags & rolling stats
6. Save final features file

### Cell 1 — Setup & configuration

In [14]:

import pandas as pd
import numpy as np
from pathlib import Path

MASTER_PATH = "master_kaligandaki_daily_withrain.csv"   # or .xlsx
FEATURES_OUT = "features_daily.csv"

START = pd.Timestamp("2019-01-01")
END   = pd.Timestamp("2023-12-31")

print("MASTER_PATH:", MASTER_PATH)


MASTER_PATH: master_kaligandaki_daily_withrain.csv


### Cell 2 — Load master (CSV/XLSX) & clean

In [15]:

def load_master(path: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Master not found: {p.resolve()}")
    if p.suffix.lower() in [".xlsx", ".xls"]:
        df = pd.read_excel(p)
    else:
        df = pd.read_csv(p)
    if "date" not in df.columns:
        for cand in ["Date","DATE","day","Day","dt","timestamp","Timestamp"]:
            if cand in df.columns:
                df = df.rename(columns={cand: "date"})
                break
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.normalize()
    df = df.dropna(subset=["date"]).drop_duplicates(subset=["date"]).sort_values("date").reset_index(drop=True)
    return df

df = load_master(MASTER_PATH)
print("Loaded rows:", len(df), "| range:", df["date"].min().date(), "→", df["date"].max().date())
df.head()


Loaded rows: 1826 | range: 2019-01-01 → 2023-12-31


,date,discharge_cms,gauge_m,rainfall_mm,peak_load_mw,avg_load_mw,energy_mwh,year
0,2019-01-01,144.0,0.80,0.0,1392.305,976.110984,29859.536,2019.0
1,2019-01-02,153.0,0.85,0.0,1413.810,938.864677,28720.160,2019.0
2,2019-01-03,150.0,0.83,0.0,1418.065,959.386210,29347.920,2019.0
3,2019-01-04,151.0,0.84,0.0,1415.075,985.114097,30134.944,2019.0
4,2019-01-05,130.0,0.71,0.0,1368.155,1011.951355,30955.904,2019.0


### Cell 3 — Standardize columns & coerce numeric

In [16]:

rename_map = {
     "discharge_cms": "discharge_m3s",  # cms == m^3/s
    "gauge_m":       "reservoir_m",
    "peak_load_mw":  "load_MW"       # use peak as target (or switch to avg_load_mw if you prefer)
    
}

df = df.rename(columns=rename_map)

for col in ["rainfall_mm","discharge_m3s","reservoir_m","load_MW"]:
    if col not in df.columns:
        df[col] = np.nan
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df[(df["date"]>=START) & (df["date"]<=END)].copy()
print(df.dtypes)
df.head(3)


date             datetime64[ns]
discharge_m3s           float64
reservoir_m             float64
rainfall_mm             float64
load_MW                 float64
avg_load_mw             float64
energy_mwh              float64
year                    float64
dtype: object


,date,discharge_m3s,reservoir_m,rainfall_mm,load_MW,avg_load_mw,energy_mwh,year
0,2019-01-01,144.0,0.80,0.0,1392.305,976.110984,29859.536,2019.0
1,2019-01-02,153.0,0.85,0.0,1413.810,938.864677,28720.160,2019.0
2,2019-01-03,150.0,0.83,0.0,1418.065,959.386210,29347.920,2019.0


In [17]:
df = df.drop(columns=["discharge_cms","gauge_m","peak_load_mw"], errors="ignore")

### Cell 4 — Continuous daily index & gap filling

In [18]:

full_index = pd.DataFrame({"date": pd.date_range(df["date"].min(), df["date"].max(), freq="D")})
df = full_index.merge(df, on="date", how="left")

df["rainfall_mm"] = df["rainfall_mm"].fillna(0)

for col in ["discharge_m3s","reservoir_m"]:
    df[col] = df[col].interpolate(limit=3, limit_direction="both")

print("NaNs remaining:")
print(df.isna().sum())
df.head(3)


NaNs remaining:
date              0
discharge_m3s    23
reservoir_m      23
rainfall_mm       0
load_MW          56
avg_load_mw      56
energy_mwh       56
year             56
dtype: int64


,date,discharge_m3s,reservoir_m,rainfall_mm,load_MW,avg_load_mw,energy_mwh,year
0,2019-01-01,144.0,0.80,0.0,1392.305,976.110984,29859.536,2019.0
1,2019-01-02,153.0,0.85,0.0,1413.810,938.864677,28720.160,2019.0
2,2019-01-03,150.0,0.83,0.0,1418.065,959.386210,29347.920,2019.0


### Cell 5 — Calendar features

In [19]:

cal = pd.DataFrame({"date": df["date"]})
cal["doy"]   = cal["date"].dt.dayofyear
cal["dow"]   = cal["date"].dt.weekday
cal["month"] = cal["date"].dt.month

cal["doy_sin"] = np.sin(2*np.pi*cal["doy"]/365.25)
cal["doy_cos"] = np.cos(2*np.pi*cal["doy"]/365.25)

df = df.merge(cal, on="date", how="left")
df.head(3)


,date,discharge_m3s,reservoir_m,rainfall_mm,load_MW,avg_load_mw,energy_mwh,year,doy,dow,month,doy_sin,doy_cos
0,2019-01-01,144.0,0.80,0.0,1392.305,976.110984,29859.536,2019.0,1,1,1,0.017202,0.999852
1,2019-01-02,153.0,0.85,0.0,1413.810,938.864677,28720.160,2019.0,2,2,1,0.034398,0.999408
2,2019-01-03,150.0,0.83,0.0,1418.065,959.386210,29347.920,2019.0,3,3,1,0.051584,0.998669


### Cell 6 — Lags & rolling stats

In [20]:

def add_lags_rolls(base: pd.DataFrame, cols, lags=(1,2,3,7,14), rolls=(3,7,14,30)) -> pd.DataFrame:
    out = base.copy().set_index("date")
    for c in cols:
        if c not in out.columns: 
            continue
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
        for W in rolls:
            roll = out[c].rolling(W, min_periods=max(1, int(W*0.6)))
            out[f"{c}_rmean{W}"] = roll.mean()
            out[f"{c}_rstd{W}"]  = roll.std()
    return out.reset_index()

feature_cols = ["rainfall_mm","discharge_m3s","reservoir_m","load_MW"]
df_feat = add_lags_rolls(df, feature_cols)
df_feat.head(3)


,date,discharge_m3s,reservoir_m,rainfall_mm,load_MW,avg_load_mw,energy_mwh,year,doy,dow,...,load_MW_lag7,load_MW_lag14,load_MW_rmean3,load_MW_rstd3,load_MW_rmean7,load_MW_rstd7,load_MW_rmean14,load_MW_rstd14,load_MW_rmean30,load_MW_rstd30
0,2019-01-01,144.0,0.80,0.0,1392.305,976.110984,29859.536,2019.0,1,1,...,NaN,NaN,1392.3050,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,153.0,0.85,0.0,1413.810,938.864677,28720.160,2019.0,2,2,...,NaN,NaN,1403.0575,15.206331,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,150.0,0.83,0.0,1418.065,959.386210,29347.920,2019.0,3,3,...,NaN,NaN,1408.0600,13.809101,NaN,NaN,NaN,NaN,NaN,NaN


### Cell 7 — Target helpers (if load exists)

In [21]:

if "load_MW" in df_feat.columns and df_feat["load_MW"].notna().any():
    df_feat["load_MW_diff1"]  = df_feat["load_MW"].diff(1)
    df_feat["load_MW_rmean7"] = df_feat["load_MW"].rolling(7, min_periods=4).mean()
df_feat.head(3)


,date,discharge_m3s,reservoir_m,rainfall_mm,load_MW,avg_load_mw,energy_mwh,year,doy,dow,...,load_MW_lag14,load_MW_rmean3,load_MW_rstd3,load_MW_rmean7,load_MW_rstd7,load_MW_rmean14,load_MW_rstd14,load_MW_rmean30,load_MW_rstd30,load_MW_diff1
0,2019-01-01,144.0,0.80,0.0,1392.305,976.110984,29859.536,2019.0,1,1,...,NaN,1392.3050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,153.0,0.85,0.0,1413.810,938.864677,28720.160,2019.0,2,2,...,NaN,1403.0575,15.206331,NaN,NaN,NaN,NaN,NaN,NaN,21.505
2,2019-01-03,150.0,0.83,0.0,1418.065,959.386210,29347.920,2019.0,3,3,...,NaN,1408.0600,13.809101,NaN,NaN,NaN,NaN,NaN,NaN,4.255


### Cell 8 — Final trim & save

In [22]:

warmup_days = 30
min_keep = df_feat["date"].min() + pd.Timedelta(days=warmup_days)
df_out = df_feat[df_feat["date"] >= min_keep].reset_index(drop=True)

df_out.to_csv(FEATURES_OUT, index=False)
print("Saved:", FEATURES_OUT, "| rows:", len(df_out), "| cols:", len(df_out.columns))
df_out.head()


Saved: features_daily.csv | rows: 1796 | cols: 66


,date,discharge_m3s,reservoir_m,rainfall_mm,load_MW,avg_load_mw,energy_mwh,year,doy,dow,...,load_MW_lag14,load_MW_rmean3,load_MW_rstd3,load_MW_rmean7,load_MW_rstd7,load_MW_rmean14,load_MW_rstd14,load_MW_rmean30,load_MW_rstd30,load_MW_diff1
0,2019-01-31,120.0,0.64,0.0,1428.760,946.092065,28941.248,2019.0,31,3,...,1315.255,1383.411667,46.998902,1331.157857,108.491752,1341.376429,94.707154,1370.451167,73.022745,42.205
1,2019-02-01,132.0,0.72,0.0,1437.155,970.446968,29686.272,2019.0,32,4,...,1407.715,1417.490000,27.117332,1337.121429,113.637486,1343.479286,96.601324,1371.229333,73.622655,8.395
2,2019-02-02,131.0,0.71,0.0,1395.755,967.499629,29596.112,2019.0,33,5,...,1374.940,1420.556667,21.885178,1341.918571,115.563832,1344.966071,97.280574,1370.485667,73.244961,-41.400
3,2019-02-03,131.0,0.71,0.0,1352.055,926.954500,28355.824,2019.0,34,6,...,1396.330,1394.988333,42.555180,1348.210714,114.604984,1341.803571,96.195958,1368.385000,72.824541,-43.700
4,2019-02-04,134.0,0.73,0.0,1373.905,918.874032,28108.640,2019.0,35,0,...,1343.085,1373.905000,21.850000,1387.015000,37.508280,1344.005000,96.579431,1368.576667,72.831481,21.850
